## Check initial WH events
This script checks the initial setup and show face events and modifies. The result from
`*_events.tsv` is stored in a file of the form `*_events_temp1.tsv`.

**Initial checks and actions:**

1. An initial `setup` event should be removed if present.
2. The initial setup event in the file should be either `setup_left_sym` (`trigger` 101) or
`setup_right_sym` (`trigger` 102).  It should be the first event.
3. The initial setup event should have sample number 1 and onset 1/srate.  (The srate = 1100).
4. The initial face event has `event_type` equal to `show_face_initial`.
It should be the second event in the file.
5. The `trigger` values for the `show_face_initial` depend on `face_type`:
    1. `famous_face`: 2
    2. `unfamiliar_face`: 3
    3. `scrambled_face`: 4


In [1]:
from hed.tools.io_utils import get_file_list, make_file_dict
from hed.tools.data_utils import get_new_dataframe
from hed.tools.hed_logger import HedLogger

bids_root_path = 'G:/WH_working2'
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events")
file_dict = make_file_dict(bids_files, indices=(0, -2))
srate = 1100

print(f"\nBIDS form of the events: {len(file_dict)}")
status = HedLogger()
for key, file in file_dict.items():
    df = get_new_dataframe(file)
    # Handle the setup event
    if df.loc[df.index[0], 'event_type'] == 'setup':
        df.drop([0], inplace=True)
        status.add(key, 'Dropped initial setup event')
    if df.loc[df.index[0], 'event_type'] == 'left_sym':
        df.loc[df.index[0], 'event_type'] = 'setup_left_sym'
        df.loc[df.index[0], 'trigger'] = 101
        status.add(key, 'Replaced left_sym with setup_left_sym and trigger 101')
    elif df.loc[df.index[0], 'event_type'] == 'right_sym':
        df.loc[df.index[0], 'event_type'] = 'setup_right_sym'
        df.loc[df.index[0], 'trigger'] = 102
        status.add(key, 'Replaced right_sym with setup_right_sym and trigger 102')
    else:
        status.add(key, f"ERROR {key}: no initial setup event in file", also_print=True)
        continue
    df.loc[df.index[0], 'sample'] = 1.0
    df.loc[df.index[0], 'onset'] = 1.0/srate

    # Now handle the initial face events
    if df.loc[df.index[1], 'event_type'] == 'show_face':
        df.loc[df.index[1], 'event_type'] = 'show_face_initial'
        status.add(key, 'Replaced event 2 show_face with show_face_initial')
    if df.loc[df.index[1], 'event_type'] != 'show_face_initial':

        status.add(key, f"ERROR {key}: missing initial show face event", also_print=True)
        continue
    if df.loc[df.index[1], 'face_type'] == 'famous_face':
        df.loc[df.index[1], 'trigger'] = 2
        status.add(key, 'Set trigger for initial famous face to 2')
    elif df.loc[df.index[1], 'face_type'] == 'unfamiliar_face':
        df.loc[df.index[1], 'trigger'] = 3
        status.add(key, 'Set trigger for initial unfamiliar face to 3')
    elif df.loc[df.index[1], 'face_type'] == 'scrambled_face':
        df.loc[df.index[1], 'trigger'] = 4
        status.add(key, 'Set trigger for initial scrambled face to 4')
    else:
        status.add(key, f"ERROR {key}: initial show face event has wrong face type {df.loc[df.index[1], 'face_type']}",
                   also_print=True)
        continue

    filename = file[:-4] + "_temp1.tsv"
    df.to_csv(filename, sep='\t', index=False)


BIDS form of the events: 108


In [2]:
status.print_log()

sub-002_run-1
	Dropped initial setup event
	Replaced right_sym with setup_right_sym and trigger 102
	Replaced event 2 show_face with show_face_initial
	Set trigger for initial unfamiliar face to 3
sub-002_run-2
	Dropped initial setup event
	Replaced right_sym with setup_right_sym and trigger 102
	Replaced event 2 show_face with show_face_initial
	Set trigger for initial scrambled face to 4
sub-002_run-3
	Dropped initial setup event
	Replaced right_sym with setup_right_sym and trigger 102
	Replaced event 2 show_face with show_face_initial
	Set trigger for initial unfamiliar face to 3
sub-002_run-4
	Dropped initial setup event
	Replaced right_sym with setup_right_sym and trigger 102
	Replaced event 2 show_face with show_face_initial
	Set trigger for initial unfamiliar face to 3
sub-002_run-5
	Dropped initial setup event
	Replaced right_sym with setup_right_sym and trigger 102
	Replaced event 2 show_face with show_face_initial
	Set trigger for initial unfamiliar face to 3
sub-002_run-6
	D